# Pangenomics
--------------------------------------------

# Reading Mapping with vg

## Overview

VG will allow you to map reads to pangenomic graphs. You will map reads from SK1 to the yeast pangenomic graph that you made with PGGB.

## Learning Objectives
+ Understanding the difference between read mapping with a reference genome versus a pangenome
+ Learn how to map reads to a pangenomic graph using vg

## Get Started

In this submodule you will be introduced to read mapping against a pangenomic graph. map SK1 reads to the PGGB yeast pangenomic graph using `vg be introduced to the Variation Graph Toolkit (VG toolkit) and learn how to use it to index graphs and get graph statistics.

#### Read Mapping
- Learn about the differences between aligning to a reference genome versus a pangenome
- Learn about alignment formats
- Get Illumina reads
- Align reads
- Get alignment statistics
- Translate ("surject") aligments into genome coordinate space 

----------------------

## Aligning to a reference genome vs a pangenome

We will align reads to the indexed pangenomic graph that you created.

Traditionally, reads are aligned to a reference genome. The reference genome represents a single individual of the species and it might be missing some of the genetic variation in the species. This means that some reads, containing this novel variation, might not align to the reference. In addition, there might be some bias when aligning reads and calling variants, with reads from individuals that are close to reference aligning better than those from more divergent individuals. This read alignment bias trickles down to the variant calling phase, possibly resulting in some missed variant calls for more evolutionarily divergent individuals.

Pangenomics graphs capture more of the genetic variation that is in species. Therefore, using them as a reference reduces issues of missing variation and reference bias.

----------------------

## Graph Alignment/Map (GAM) Format

[GAM](https://github.com/vgteam/vg/wiki/File-Formats) is an alignment format analogous to [BAM](https://samtools.github.io/hts-specs/SAMv1.pdf), but for graphs.  
+ Binary file describing where reads mapped to in the graph structure  
+ Uncompressed has one read per line  
+ Can be converted to JSON for manual parsing (very inefficient!)

----------------------

## Get Reads for Mapping

We will use paired-end Illumina reads from SK1, which was also included in our graph. You could also align reads from an accession that is not in our graph. Download them using sratoolkit's `prefetch` and `fasterq-dump` commands from the Short Read Archive (SRA). They are in run accession SRR4074258.

1. First, `prefetch` the accession, which makes getting the fastq data faster.

The parameters:

`--output-directory` specify output directory

In [ ]:
!prefetch SRR4074258 --output-directory reads

2. Then get the fastq files. `fasterq-dump` allows us to multi-thread the download and will automatically put the read1 and read2 sequence in different files. Point it to the prefetched data (SRR4074258/SRR4074258.sra).

The parameters:

`--outdir`  output directory ("." indicates the current directory)  
`--outfile`  output file name (fasterq-dump will add _1 and _2 before the fastq as it separates out read1 and read2)  
`--threads`  number of threads  
`--progress`  show progress

In [ ]:
!fasterq-dump --outdir reads --outfile SK1.illumina.fastq --threads 4 --progress reads/SRR4074258/SRR4074258.sra

3. Now we have to *zip* the files, and rather than using `gzip`, we will use `pigz` (**p**arallel **i**mplementation of **gz**ip") so that we can use multiple threads and complete the process more quickly. By default it will use all available threads, though that can be adjusted with the `--processes` parameter.

<div class="alert alert-block alert-info"> <b>NOTE:</b> The verbose setting (`-v`) is not very verbose so make sure you wait until the asterisk in the square brackets to the left of the code block is replaced by a number to know that it is done.

In [ ]:
!pigz reads/SK1.illumina_1.fastq
!pigz reads/SK1.illumina_2.fastq

4. Next, remove the prefetch directory.

In [ ]:
!rm -r reads/SRR4074258

----------------------

## Read Mapping

1. Make an alignments directory.


In [ ]:
!mkdir alignments

2. We will use `vg giraffe` to map paired-end Illumina reads from the SK1 yeast accession to the chrVIII graph (yprp.chrVIII.pggb.vg).

The parameters:  

`-Z`  .gbz file  
`-m`  .min file  
`-d`  .dist file
`-f`  fastq file (use more than once if feeding in paired-end read files)

We will redirect the output into a gam file.

In [ ]:
!vg giraffe -p -Z graphs/yprp.chrVIII.pggb.giraffe.gbz -m graphs/yprp.chrVIII.pggb.min -d graphs/yprp.chrVIII.pggb.dist -f reads/SK1.illumina_1.fastq.gz -f reads/SK1.illumina_2.fastq.gz > alignments/SK1xyprp.chrVIII.pggb.mapped.gam

<div class="alert alert-block alert-success"> <b>Try this in the cells below:</b>  
    <ul>
        <li>Align the SK1 reads to the graph created directly from the full genome (yprp.fullgenome.pggb.giraffe.gbz).</li>
        <li>Align the SK1 reads to the graph created by combining the 16 chromosomal subgraphs (yprp.allchrs.pggb.giraff.gbz).</li>
    </ul>

In [ ]:
# Align the SK1 reads to the genome graph created directly from the full genome

In [ ]:
# Align the SK1 reads to the genome graph created by combining the 16 chromosomal subgraphs

<details>
<summary>Click for help</summary>
<br>
!vg giraffe -p -Z graphs/yprp.fullgenome.pggb.giraffe.gbz -m graphs/yprp.fullgenome.pggb.min -d graphs/yprp.fullgenome.pggb.dist -f reads/SK1.illumina_1.fastq.gz -f reads/SK1.illumina_2.fastq.gz > alignments/SK1xyprp.fullgenome.pggb.mapped.gam  


!vg giraffe -p -Z graphs/yprp.allchrs.pggb.giraffe.gbz -m graphs/yprp.allchrs.pggb.min -d graphs/yprp.allchrs.pggb.dist -f reads/SK1.illumina_1.fastq.gz -f reads/SK1.illumina_2.fastq.gz > alignments/SK1xyprp.allchrs.pggb.mapped.gam
</details>

----------------------

## Mapping statistics

1. Now we can compute some mapping statistics using `vg stats`.

The parameters:

`-a` alignment (GAM) file

In [ ]:
!vg stats -a alignments/SK1xyprp.chrVIII.pggb.mapped.gam

The first two lines reflect the number of reads.

Run the code below to see the flashcard.

In [ ]:
from IPython.display import IFrame
IFrame('../html/flashcard_readalign.html', width=800, height=400)

<div class="alert alert-block alert-success"> <b>Try this in the cells below:</b>  
    <ul>
        <li>Get alignment stats for the graph created directly from the full genome (yprp.fullgenome.pggb.vg).</li>
        <li>Get alignment stats for the graph created by combining the 16 chromosomal subgraphs (yprp.allchrs.pggb.vg).</li>
        <li>What differences do you notice between the stats for these two graphs?</li> 
    </ul>

In [ ]:
# Get alignment stats for the genome graph created directly from the full genome

In [ ]:
# Get alignment stats for the genome graph created by combining the 16 chromosomal subgraphs

<details>
<summary>Click for help</summary>
<br>
!vg stats -a alignments/SK1xyprp.fullgenome.pggb.mapped.gam  

!vg stats -a alignments/SK1xyprp.allchrs.pggb.mapped.gam
</details>

In [ ]:
from IPython.display import IFrame
IFrame('../html/flashcard_aligndiff.html', width=800, height=400)

----------------------

## Bringing Alignments Back to Individual Genomes

1. Our reads are mapped to the pangenomic graph. If we need to bring the alignments back into coordinates for individual genomes, we can "surject" them into a genome of our choice using `vg surject`.

The parameters:

`-i` pairs are interleaved
`-x` the graph to use  
`-b` stdout is in bam format  
`-t` the number of threads to use  
`-p` path to surject into  
Graph alignment file (gam)

We will redirect the output into a bam file.

We'll surject variants onto S288C_chrVIII.

In [ ]:
!vg surject -i -x graphs/yprp.chrVIII.pggb.giraffe.gbz -b -t 20 -p S288C_chrVIII alignments/SK1xyprp.chrVIII.pggb.mapped.gam > alignments/SK1xyprp.chrVIII.pggb.mapped.bam

2. Let's get some alignment statistics from the bam file using `samtools stats`. We will focus on the summary numbers.

In [ ]:
!samtools stats alignments/SK1xyprp.chrVIII.pggb.mapped.bam | grep ^SN | cut -f 2-

Run the code below to show flashcards about the alignment statistics.

In [ ]:
from IPython.display import IFrame
IFrame('../html/flashcard_bam.html', width=800, height=400)

Now, surject it for the entire S288C genome using the alignments in SK1xyprp.fullgenome.pggb.mapped.gam.

The `-p` parameter will only take one sequence. We will use the `-F` parameter because it allows you to feed in a file with a list of sequences.

3. First, make a file with a list of sequences from the S288C genome.

In [ ]:
!grep '>S288C' assemblies/yprp.all.fa | sed 's/>//' > assemblies/S288C.seqs.txt

4. Next, run the surjection.

In [ ]:
!vg surject -i -x graphs/yprp.fullgenome.pggb.giraffe.gbz -b -t 20 -F assemblies/S288C.seqs.txt alignments/SK1xyprp.fullgenome.pggb.mapped.gam > alignments/SK1xyprp.fullgenome.pggb.mapped.bam

<div class="alert alert-block alert-success"> <b>Try this in the cell below:</b>  
    <ul>
        <li>Get alignment stats for the S288C variants surjected from the full graph .gam file (alignments/SK1xyprp.fullgenome.pggb.mapped.bam)</li>
    </ul>

In [ ]:
# Get alignment stats for the S288C variants surjected from the full graph

<details>
<summary>Click for help</summary>
<br>

!samtools stats alignments/SK1xyprp.fullgenome.pggb.mapped.bam | grep ^SN | cut -f 2-

</details>

----------------------

## Conclusion

In this submodule, you learned how to align reads directly to a pangenomic graph and how to surject those alignments into coordinates within individual genomes. In addition, you learned how to calculate alignment statistics. We also discussed details about the SAM, BAM, and GAM alignment file formats.

----------------------

## Clean up

<div class="alert alert-warning">No cleanup is necessary for this submodule. Don't forget to shutdown your Workbench when you are done working through this module!.</div>